# PROJETO ETL - Análise do desempenho dos alunos do Ceará no ENEM 2019

## PARTE I - Extraindo os dados

In [ ]:
#Instalação/Aualizaç da bilbioteca dask para gerenciamento de datframe de grandes tamanhos. Será Utilizada ao invés do Pandas 
#devido ao tamanho do arquivo .csv
!pip install dask
!pip install --upgrade pandas

### 1.0 - Importação de bibliotecas Úteis

In [23]:
#importação de Bibliotecas
import dask.dataframe as dd
import pandas as pd
import zipfile
import requests
import os
from io import BytesIO

### 2.0 - Preparação do Ambiente (Criação de Pasta) e Efetuando Download do Arquivo do Site do Inep

In [4]:
#Criar estrutura de pastas para armazenamento do arquivo baixado
os.makedirs('./enem2019', exist_ok=True)

In [5]:
#Baixando o arquivo do link abaixo e descompactando na pasta /enem2019
#URL dos dados
url = 'https://download.inep.gov.br/microdados/microdados_enem_2019.zip'
#Baixando o Arquivo .ZIP
print('Baixando o Arquivo....')
filebytes = BytesIO(
    requests.get(url).content
)
#Descompatando na pasta destino
print('Download Concluído. Aguarde a descompactação....')
myzip = zipfile.ZipFile(filebytes).extractall('./enem2019')
print('Processamento Concluído.')

### 3.0 - Lendo os dados em lote (Arquivo Muito Grande) e colocando em um dataframe já Filtrado por Estado = Ceará

In [2]:
# Cria dataframe com mais memória devido ao tamanho do arquivo
enem = dd.read_csv(
    './enem2019/DADOS/microdados_enem_2019.csv',
    sep = ';' , 
    decimal=',' ,
    dtype={'CO_MUNICIPIO_NASCIMENTO': 'float64',
            'CO_UF_NASCIMENTO': 'float64',
            'NU_IDADE': 'float64',
            'NO_MUNICIPIO_ESC': 'object',
            'SG_UF_ESC': 'object'},
    encoding='latin1'
)

In [3]:
#Recriando o Dataframe Só para Ceará = Processo de Filtragem
enem = enem.loc[enem.SG_UF_RESIDENCIA == 'CE']

## PARTE II - Transformando os dados

In [12]:
#Recriando o Dataframe só com as colunas necessárias
enem_ce = enem[['NU_INSCRICAO',         
                'SG_UF_RESIDENCIA',     
                'TP_SEXO',
                'TP_COR_RACA',
                'TP_ESCOLA',
                'TP_ENSINO',
                'NU_NOTA_CN',
                'NU_NOTA_MT',
                'Q001',
                'Q002',
                'Q004',
                'Q005',
                'Q006']].compute()

In [31]:
#Analisando Quantidade de Nulos no Dataframe
enem_ce.isnull().sum()

NU_INSCRICAO            0
SG_UF_RESIDENCIA        0
TP_SEXO                 0
TP_COR_RACA             0
TP_ESCOLA               0
TP_ENSINO           75865
NU_NOTA_CN              0
NU_NOTA_MT              0
Q001                    0
Q002                    0
Q004                    0
Q005                    0
Q006                    0
dtype: int64

In [15]:
#Cria backuo do DataFrame
Enem_bkp = enem_ce

In [29]:
#Apagando as Linhas com Notas Zeradas - Excluindos Alunos Faltantes
enem_ce = enem_ce.dropna(subset=['NU_NOTA_MT'])

In [30]:
#Quantidade de Linhas
enem_ce.shape[0]

222780

In [38]:
#Verificando os Tipos de Dados do Dataframe
enem_ce.dtypes

NU_INSCRICAO          int64
SG_UF_RESIDENCIA     object
TP_SEXO              object
TP_COR_RACA           int64
TP_ESCOLA             int64
TP_ENSINO           float64
NU_NOTA_CN          float64
NU_NOTA_MT          float64
Q001                 object
Q002                 object
Q004                 object
Q005                  int64
Q006                 object
dtype: object

In [37]:
#Atualizando Tipos de Dados de Colunas Importantes
enem_ce['NU_NOTA_CN'] = enem_ce['NU_NOTA_CN'].astype('float')
enem_ce['NU_NOTA_MT'] = enem_ce['NU_NOTA_MT'].astype('float')

In [39]:
#Analise detalhada do Dataframe
enem_ce.describe()

,NU_INSCRICAO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,NU_NOTA_CN,NU_NOTA_MT,Q005
count,2.227800e+05,222780.000000,222780.000000,146915.000000,222780.000000,222780.000000,222780.000000
mean,1.900030e+11,2.443016,1.461379,1.002001,468.256969,517.529065,4.105333
std,1.408311e+06,0.956225,0.573180,0.044690,75.815679,109.034242,1.450371
min,1.900010e+11,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,1.900019e+11,2.000000,1.000000,1.000000,409.300000,431.900000,3.000000
50%,1.900028e+11,3.000000,1.000000,1.000000,457.500000,492.700000,4.000000
75%,1.900042e+11,3.000000,2.000000,1.000000,520.800000,587.800000,5.000000
max,1.900061e+11,5.000000,3.000000,2.000000,841.800000,985.500000,20.000000


## PARTE III - Carregamento dos dados